In [1]:
import pandas as pd 

data = pd.read_csv("data_experiments.csv")
data.head()

,doc_id,url,cache,fulltext,nature,published,entity_name,entity_type,geo_path,extracted_text
0,40369/749929d81bfa6b074fe6844e86b36dec643747bd...,https://sde09.fr/wp-content/uploads/2024/01/Bu...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,comm,2024-01-01 00:00:00,Syndicat départemental d'énergies de l'Ariège ...,Autre groupement,CC Vendée Grand Littoral/Vendée/Pays de la Loire,BULLETIN D'INFORMATION DU SYNDICAT DÉPARTEMENT...
1,2702/19edd8d6b29dcafcb309e0997ddfaef8cca75422_...,https://www.cc-paysdelimours.fr/files/CCPL-202...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,comm,2021-08-22 00:00:00,CC du Pays de Limours (CCPL),Intercommunalité,CC du Pays de Limours (CCPL)/Essonne/Île-de-Fr...,Communauté de communes du Pays de Limours PCAE...
2,4172/0bf2fa62554e88e675252e51c5720d874aea4c5d_...,https://www.mairie-orly.fr/content/download/16...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,comm,2024-03-21 00:00:00,Orly,Commune,Orly/Métropole du Grand Paris (94)/Val-de-Marn...,AccessDeniedAccess Denied.tx2fe52a9984db4526a1...
3,2774/25ae7_DEL-2022-199_Annexe_Bilan_dactivite...,https://www.grandannecy.fr/fileadmin/mediatheq...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,comm,2022-09-01 00:00:00,CA du Grand Annecy,Intercommunalité,CA du Grand Annecy/Haute-Savoie/Auvergne-Rhône...,H O T E L SOM MAIRE P. 4 administration généra...
4,6472/022f3bc3b0e35408bde88d61f3049899113f21e0_...,https://ville-somain.fr/wp-content/uploads/202...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,comm,2021-10-01 00:00:00,Somain,Commune,Somain/CC Cœur d'Ostrevent/Nord/Hauts-de-France,N°42 SEPTEMBRE-OCTOBRE 2020 Création de la bal...


In [2]:
data.iloc[3]["url"]

'https://www.grandannecy.fr/fileadmin/mediatheque/Documents_reglementaires/Conseil/2022/09_29/DEL-2022-199_Annexe_Bilan_dactivite_2021_du_Grand_Annecy.pdf'

In [3]:
data.shape

(2000, 10)

## Automatic Data Annotation with Transformers and Mixtral 7B

In [4]:
with open("HF_TOKEN.txt", "r") as f:
        HF_TOKEN = f.read()

In [5]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

#### Prompt Functions WITH TRANSFORMERS

Important note: the API only supports a context length of 8k for this model, while the model may support 128k. Some random dude got around the problem by running a private endpoint and changing the 'Container Configuration', specifically the token settings to whatever length they required.

Apparent solution PAID: This part is relatively straightforward. Go to the the model card (e.g. https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct), Click on "Deploy" in the top right corner and select "Inference Endpoint". In the next page you can choose what hardware you want to run the model on, which will impact how much you will pay per hour. Set "Automatic Scale to Zero" to some value other than "never" to switch off the endpoint after X amount of time without request, so that you won't be paying for the endpoint while it's not in use. Then go to "Advanced Configuration" and set the maximum amount of tokens to whatever makes sense for your use case. With this procedure you will be able to make full use of the larger context windows of Llama 3 models.




In [6]:
def lazy_length_check(text, limit=8000, base_prompt_len=888):# 888 is the length of the current prompt

    chars = text.split(" ")
    len_text = len(chars)
    if len_text > limit - base_prompt_len: 
        print("Text over max. length. Applying padding.")
        padded_text = " ".join(chars[:limit - base_prompt_len])
        return padded_text 
    
    else:
        return text

In [7]:
def generate_prompt(text_to_classify):
    base_prompt = f"""<OBJECTIF_ET_PERSONA>
    Vous êtes un annotateur manuel. Votre tâche consiste à dire si un INPUT TEXT concerne un projet ou pas.
    </OBJECTIF_ET_PERSONA>

    <INSTRUCTIONS>
    Suivre ces étapes :
    1. Déterminez si le INPUT TEXT concerne un projet ou non.
    2. Si le texte concerne un projet, renvoyez 1. Si le INPUT TEXT ne concerne pas un projet, renvoyez 0.
    </INSTRUCTIONS>

    <FORMAT_DE_SORTIE>
    Il doit que votre réponse est un numéro unique: 0 ou 1.
    </FORMAT_DE_SORTIE>

    <EXEMPLES>
    1. INPUT TEXT: "Pas moins de 200 tableaux peints par les impressionnistes à Bougival ont été répertoriés. Nombreux furent les peintres à s'installer pour un temps à Bougival ou à poser leur chevalet sur les rives de la Seine pour immortaliser paysage et vie en bord du fleuve. Lebourg (Alfred) 1849 – 1928 : attiré par la vallée de la Seine, Albert Lebourg peint à de nombreuses reprises le quai qui borde la Seine venant de Rueil. Il peint Bord de la Seine à Bougival en 1885. Monet (Claude) 1840 – 1926 : il s'installe à Bougival en 1869. Avec son tableau La Seine à Bougival ou le Pont de Bougival peint vers 1869/1870, Claude Monet a représenté l’ancien pont de la ville qui était à péage. Morisot (Berthe) 1841 – 1895 : elle passe plusieurs étés à Bougival de 1881 à 1884 dans la maison qui va devenir, en 2024, l’espace muséal Berthe Morisot. Elle peint à Bougival une quarantaine de toiles : La Fable (1883), Le Quai à Bougival (1883), Eugène Manet et sa fille dans le jardin (1883), Dans la Véranda (1884), Jardin à Bougival (1884), Roses trémières (1884), etc. Pissarro (Camille) 1830 – 1903 : installé à Louveciennes puis à Pontoise, Pissarro peint plusieurs toiles à Bougival : Maisons à Bougival (1870), Bords de la Seine à Bougival (1871), La Route de Louveciennes (1872). Renoir (Auguste) 1849 – 1928 : Renoir n’a jamais séjourné à Bougival, contrairement à son ami Monet, mais"
    0

    2. INPUT TEXT: "AMBITIONS DE LA ZAC : Une réponse aux enjeux contemporains Construit sur une friche industrielle et au contact d’un futur parc et du grand paysage, ce nouveau quartier répond à diérents enjeux : > Sociétaux : • Répondre aux besoins en logements de la commune, • Proposer un quartier avec une densité dénie pour ralentir l’étalement urbain, • Orir un accès aux services, espaces de santé, commerces et transports en commun, • Desservir le quartier par le réseau de transports en commun de la métropole (prolongation de la ligne 55), • Conserver l’identité culturelle et patrimoniale des lieux grâce au maintien des éléments industriels du site. > Environnementaux : • Renforcer la biodiversité de ce terrain malmené depuis 60 ans, en intégrant des plantations résistantes aux fortes chaleurs et la perméabilisation du site à 50 %. • Dépolluer les sols, • Respecter la charte Écoquartier : obtension de la médaille d’argent QDO en phase conception (Quartier Durable d’Occitanie). • Construire des bâtiments respectant les normes environnementales en vigueur, • Favoriser les modes de transport alternatifs à la voiture : création de pistes cyclables et de cheminements doux, d’une centralité orant des commerces et des activités, un accès à moins de 10 minutes à pied à la rive droite de la commune, • Préserver la limite entre ville et plaine cultivée. • Protéger les spots de biodiversités identiés dans l’étude d’impact (environ 7000 m2 ), • Orir un espace public paysager qualitatif avec près d’1 ha de parc urbain et de venelles vertes."
    1

    <INPUT TEXT>
    {text_to_classify}
    </INPUT TEXT>
    """
    return base_prompt

In [8]:
def format_prompt(prompt):
    messages = [{"role": "user", "content": prompt}]
    messages_tokenized = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, return_tensors="pt"
    )
    return messages_tokenized

In [9]:
generation_params = dict(
    temperature=0.1,
    top_p=0.60,
    top_k=None,
    repetition_penalty=1.0,
    do_sample=True,
    max_new_tokens=150,
    return_full_text=False,
    seed=42,
    max_time=None,
    stream=False,
    use_cache=False,
    wait_for_model=False,
)

#### Querying

In [10]:
import requests
import huggingface_hub

def query(payload=None, api_url=None):
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

headers = {"Authorization": f"Bearer {huggingface_hub.get_token()}"}
api_url = "https://api-inference.huggingface.co/models/" + model_id

In [11]:
# def clean_response(response):
    
#     list_bad_chars = ["\\", "\n", "\t"]
#     for bad_char in list_bad_chars:
#         while bad_char in response:
#             response = response.replace(bad_char, "")
    
#     # if response.count("\"") != 6:
#     #     response = response + "\""
#     if "}" not in response:
#         response = response + "}"
#     return response

In [12]:
import re

def parse_response(output):
    
    regex_expression = r"[01]" # try parsing reasons in the future?
    regex = re.compile(regex_expression)
    match = re.search(regex, output)
    # print(match)
    parsed_response = match.group()
    # print(parsed_response)
    return parsed_response

In [13]:
def llm_annotate(text, verbose=True):
    
    text = lazy_length_check(text)
    
    prompt = generate_prompt(text)
    prompt_formated = format_prompt(prompt)
    
    output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)
    if verbose:
        print(output)
    
    annotation = parse_response(output[0]["generated_text"])
    
    return annotation

In [14]:
# data["mistral_annotation"] = data["extracted_text"].apply(llm_annotate)

#### Example

In [15]:
# text = lazy_length_check(example_text)

# prompt = generate_prompt(text)
# prompt_formated = format_prompt(prompt)

# output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)

In [16]:
# output

In [17]:
# response = output[0]["generated_text"]
# print(type(response))
# response

In [18]:
# new_response = clean_response(response)
# new_response

In [19]:
# parsed_response = parse_response(new_response)

In [20]:
# def llm_correct_json_prompt(json_dict):
#     prompt_correct_format =f"""Vous êtes programmeur. Vous corrigez les erreurs des étudiants lorsqu’ils formatent manuellement les dictionnaires JSON.
#     Il s'agit d'un dictionnaire JSON qui n'est pas correctement corrigé:
#     {json_dict}
#     Renvoie le dictionnaire JSON formaté sur une seule ligne sans erreur.
#     <ÉTAPES À SUIVRE>
#     1. Vérifiez que toutes les chaînes sont entre guillemets.
#     2. Supprimez les retraits, les espaces et les nouvelles lignes.
#     3. Vérifiez qu'il y a : après chaque clé et , après les éléments du dictionnaire.
#     4. Renvoyez le dictionnaire JSON corrigé. Votre réponse devrait commencer directement.
#     5. Vérifiez que la response est 0 ou 1."""
    
#     return prompt_correct_format

# def llm_correct_json(prompt_correct_format):
    
#     prompt_correct_format = llm_correct_json_prompt(prompt_correct_format)
#     # print(prompt_correct_format)
#     output = query(payload={"inputs": prompt_correct_format, "parameters": {**generation_params}}, api_url=api_url)
    
#     return output[0]["generated_text"]

In [21]:
# verifying that we have the correct types to parse
# print(type(output_dictionary))
# print(type(output_dictionary["is_project"]))
# print(type(output_dictionary["reason"]))
# print(type(output_dictionary["reason"][0]))

In [22]:
# from numpy import NaN

# # data["mixtral_llm_is_project"] = NaN
# # data["mixtral_llm_reasoning"] = NaN

# # how much annotation is left
# data[data["mixtral_llm_is_project"].isna()].shape

In [23]:
# data = pd.read_csv("data_experiments_annotated.csv")

In [24]:
# import random
# from tqdm import tqdm

# tqdm.pandas()

# index_data_no_annot = list(data[data["mixtral_llm_is_project"].isna()].index)

# while len(index_data_no_annot) != 0:
    
#     print(len(index_data_no_annot), " left.")
    
#     current_index = random.choice(index_data_no_annot)
#     print("Current index ", current_index)
    
#     current_text = data.iloc[current_index]["extracted_text"]
#     print("Current text:", current_text[0:100] + "...")
    
#     output = llm_annotate(current_text)
    
#     try:
        
#         response_clean = clean_response(output)
#         is_project = parse_response(response_clean)
        
#         print("OUTPUT\n")
#         print(is_project)
#         # print(reasoning)
        
#         data.at[current_index, "mixtral_llm_is_project"] = is_project
#         # data.at[current_index, "mixtral_llm_reasoning"] = reasoning
        
#         index_data_no_annot = list(data[data["mixtral_llm_is_project"].isna()].index)
        
#     except:
        
#         print("Error parsing the LLM output.\n Calling second agent.")
#         correct_llm_prompt = llm_correct_json_prompt(output)
#         output = llm_correct_json(correct_llm_prompt)
        
#         try:
            
#             response_clean = clean_response(output)
#             is_project = parse_response(response_clean)
            
#             print("OUTPUT\n")
#             print(is_project)
#             # print(reasoning)
            
#             data.at[current_index, "mixtral_llm_is_project"] = is_project
#             # data.at[current_index, "mixtral_llm_reasoning"] = reasoning
            
#         except:
            
#             print("Second agent couldn't fix it either.")
#             print(f"Final output:\n{output}")
        
#         index_data_no_annot = list(data[data["mixtral_llm_is_project"].isna()].index)
            
#     if current_index % 5 == 0:
#         data.to_csv("data_experiments_annotated.csv", index=False)

In [25]:
# data.to_csv("data_experiments_annotated.csv", index=False)

## Data Annotation with Llama3 and LANGCHAIN

In [26]:
import pandas as pd
from numpy import NaN

data = pd.read_csv("data_experiments_annotated.csv")
data["llama3:1b_llm_is_project"] = NaN

In [27]:
data.columns

Index(['doc_id', 'url', 'cache', 'fulltext', 'nature', 'published',
       'entity_name', 'entity_type', 'geo_path', 'extracted_text',
       'llama3:1b_llm_is_project'],
      dtype='object')

In [28]:
unannotated_rows = data[data["llama3:1b_llm_is_project"].isna()].shape[0]
unannotated_rows

2000

In [29]:
base_prompt = """Vous êtes un annotateur manuel. Votre tâche consiste à dire si un INPUT TEXT concerne un projet ou pas.
    Suivre ces étapes:
    1. Déterminez si le INPUT TEXT concerne un projet ou non.
    2. Si le texte concerne un projet, renvoyez 1. Si le INPUT TEXT ne concerne pas un projet, renvoyez 0.

    Il doit que votre réponse est un numéro unique: 0 ou 1.

    EXEMPLES
    INPUT TEXT: "Pas moins de 200 tableaux peints par les impressionnistes à Bougival ont été répertoriés. Nombreux furent les peintres à s'installer pour un temps à Bougival ou à poser leur chevalet sur les rives de la Seine pour immortaliser paysage et vie en bord du fleuve. Lebourg (Alfred) 1849 – 1928 : attiré par la vallée de la Seine, Albert Lebourg peint à de nombreuses reprises le quai qui borde la Seine venant de Rueil. Il peint Bord de la Seine à Bougival en 1885. Monet (Claude) 1840 – 1926 : il s'installe à Bougival en 1869. Avec son tableau La Seine à Bougival ou le Pont de Bougival peint vers 1869/1870, Claude Monet a représenté l’ancien pont de la ville qui était à péage. Morisot (Berthe) 1841 – 1895 : elle passe plusieurs étés à Bougival de 1881 à 1884 dans la maison qui va devenir, en 2024, l’espace muséal Berthe Morisot. Elle peint à Bougival une quarantaine de toiles : La Fable (1883), Le Quai à Bougival (1883), Eugène Manet et sa fille dans le jardin (1883), Dans la Véranda (1884), Jardin à Bougival (1884), Roses trémières (1884), etc. Pissarro (Camille) 1830 – 1903 : installé à Louveciennes puis à Pontoise, Pissarro peint plusieurs toiles à Bougival : Maisons à Bougival (1870), Bords de la Seine à Bougival (1871), La Route de Louveciennes (1872). Renoir (Auguste) 1849 – 1928 : Renoir n’a jamais séjourné à Bougival, contrairement à son ami Monet, mais"
    OUTPUT: 0

    INPUT TEXT: "AMBITIONS DE LA ZAC : Une réponse aux enjeux contemporains Construit sur une friche industrielle et au contact d’un futur parc et du grand paysage, ce nouveau quartier répond à diérents enjeux : > Sociétaux : • Répondre aux besoins en logements de la commune, • Proposer un quartier avec une densité dénie pour ralentir l’étalement urbain, • Orir un accès aux services, espaces de santé, commerces et transports en commun, • Desservir le quartier par le réseau de transports en commun de la métropole (prolongation de la ligne 55), • Conserver l’identité culturelle et patrimoniale des lieux grâce au maintien des éléments industriels du site. > Environnementaux : • Renforcer la biodiversité de ce terrain malmené depuis 60 ans, en intégrant des plantations résistantes aux fortes chaleurs et la perméabilisation du site à 50 %. • Dépolluer les sols, • Respecter la charte Écoquartier : obtension de la médaille d’argent QDO en phase conception (Quartier Durable d’Occitanie). • Construire des bâtiments respectant les normes environnementales en vigueur, • Favoriser les modes de transport alternatifs à la voiture : création de pistes cyclables et de cheminements doux, d’une centralité orant des commerces et des activités, un accès à moins de 10 minutes à pied à la rive droite de la commune, • Préserver la limite entre ville et plaine cultivée. • Protéger les spots de biodiversités identiés dans l’étude d’impact (environ 7000 m2 ), • Orir un espace public paysager qualitatif avec près d’1 ha de parc urbain et de venelles vertes."
    OUTPUT: 1

    Déterminer quel devrait être le output de ce texte:
    {text_to_classify}
    """

In [30]:
base_prompt_2 ="""CE DOCUMENT PARLE-T-IL D'UN PROJET ? RÉPONDRE OUI OU NON. LE FORMAT FINAL DEVRAIT ÊTRE UN DICTIONNAIRE JSON.
INPUT TEXT:{text_to_classify}
VOTRE OUTPUT:
"""

In [31]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

llm = OllamaLLM(model="llama3.2:1b") # 128k token window size

def llm_annotate_langchain(text_to_classify, base_prompt=base_prompt):
    
    prompt_template = PromptTemplate(input_variables=["text_to_classify"],
                                    template=base_prompt)
    prompt = prompt_template.format(text_to_classify = text_to_classify)
    output = llm.invoke(prompt)
    annotation = parse_response(output)
    
    return annotation, output

example = data.iloc[0]["extracted_text"]
annotation, output = llm_annotate_langchain(example, base_prompt)
print(annotation)
print(output)


La Mission Chaleur Renouvelable (MCR) est un service proposé par l'Agence NOVA pour accompagner les habitants des communautés ariégiennes dans la rénovation énergétique. Voici quelques informations clés sur ce service :

**Objectif**

La mission vise à aider les maitres d'ouvrages àriégeois (qui possèdent un numéro SIRET) à réussir leur projet de rénovation énergétique en leur offrant des conseils techniques et financiers.

**Projets qui peuvent bénéficier du service**

Les projets qui font appel à des systèmes, chaufferies automatiques dédiées ou réseaux de production de chaleur (ou froid) à partir des énergies renouvelables thermiques sont admis. Ces projets peuvent être de type :

* Chauffage centralisé
* Chauvesource
* Réseaux de chauffe

**Avantages du service**

La Mission Chaleur Renouvelable offre plusieurs avantages aux maitres d'ouvrages, notamment :

* Des conseils techniques objectifs et gratuits pour définir le système adapté et optimiser les installations en fonctionnemen

In [32]:
# data["llama3:1b_llm_is_project"] = data["extracted_text"].apply(llm_annotate_langchain)

In [33]:
# data.to_csv("data_experiments_annotated.csv", index=False)